In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [2]:
batch_size= 64
epochs= 100
# latent dimensionality of encoding sspace
latent_dim= 256
# no of samples to train
num_samples= 1000
data_path= 'fra.txt'

In [3]:
# vectorize data
input_texts=[] # english charactres
target_texts=[] # french charactres

input_characters= set() # input words
target_characters= set() # output words

with open(data_path, 'r', encoding= 'utf-8') as f:
    lines= f.read().split('\n')

for line in lines[: min(num_samples, len(lines) -1)]:
    input_text, target_text, _= line.split('\t')
    target_text=  target_text 
    input_texts.append(target_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
            
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
input_characters

{' ',
 '!',
 '$',
 "'",
 ',',
 '.',
 '1',
 '5',
 '9',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [5]:
target_characters

{' ',
 '!',
 "'",
 ',',
 '-',
 '.',
 '1',
 '9',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '\xa0',
 'À',
 'Ç',
 'É',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'î',
 'ô',
 'ù',
 'û',
 '\u2009',
 '’',
 '\u202f'}

In [6]:
input_texts

['Va !',
 'Marche.',
 'Bouge !',
 'Salut !',
 'Salut.',
 'Cours\u202f!',
 'Courez\u202f!',
 'Prenez vos jambes à vos cous !',
 'File !',
 'Filez !',
 'Cours !',
 'Fuyez !',
 'Fuyons !',
 'Cours\u202f!',
 'Courez\u202f!',
 'Prenez vos jambes à vos cous !',
 'File !',
 'Filez !',
 'Cours !',
 'Fuyez !',
 'Fuyons !',
 'Qui ?',
 'Ça alors\u202f!',
 'À terre\xa0!',
 'Baisse-toi\xa0!',
 'Baissez-vous\xa0!',
 'Au feu !',
 "À l'aide\u202f!",
 'Cache-toi.',
 'Cachez-vous.',
 'Saute.',
 'Saute.',
 'Ça suffit\u202f!',
 'Stop\u202f!',
 'Arrête-toi !',
 'Attends !',
 'Attendez !',
 'Attendez.',
 'Attends !',
 'Attendez !',
 'Attends.',
 'Attendez.',
 'Commencez.',
 'Commence.',
 'Poursuis.',
 'Continuez.',
 'Poursuivez.',
 'Bonjour !',
 'Salut !',
 'Je comprends.',
 'Aha.',
 "J'essaye.",
 "J'ai gagné !",
 "Je l'ai emporté !",
 'J’ai gagné.',
 'Oh non !',
 'Calme-toi.',
 'Détends-toi\u202f!',
 'Détendez-vous\u202f!',
 'Relaxe, Max\u202f!',
 'Cool, Raoul\u202f!',
 'Du calme\u202f!',
 'Relaxe !',
 'Ca

In [7]:
target_texts

['Va !',
 'Marche.',
 'Bouge !',
 'Salut !',
 'Salut.',
 'Cours\u202f!',
 'Courez\u202f!',
 'Prenez vos jambes à vos cous !',
 'File !',
 'Filez !',
 'Cours !',
 'Fuyez !',
 'Fuyons !',
 'Cours\u202f!',
 'Courez\u202f!',
 'Prenez vos jambes à vos cous !',
 'File !',
 'Filez !',
 'Cours !',
 'Fuyez !',
 'Fuyons !',
 'Qui ?',
 'Ça alors\u202f!',
 'À terre\xa0!',
 'Baisse-toi\xa0!',
 'Baissez-vous\xa0!',
 'Au feu !',
 "À l'aide\u202f!",
 'Cache-toi.',
 'Cachez-vous.',
 'Saute.',
 'Saute.',
 'Ça suffit\u202f!',
 'Stop\u202f!',
 'Arrête-toi !',
 'Attends !',
 'Attendez !',
 'Attendez.',
 'Attends !',
 'Attendez !',
 'Attends.',
 'Attendez.',
 'Commencez.',
 'Commence.',
 'Poursuis.',
 'Continuez.',
 'Poursuivez.',
 'Bonjour !',
 'Salut !',
 'Je comprends.',
 'Aha.',
 "J'essaye.",
 "J'ai gagné !",
 "Je l'ai emporté !",
 'J’ai gagné.',
 'Oh non !',
 'Calme-toi.',
 'Détends-toi\u202f!',
 'Détendez-vous\u202f!',
 'Relaxe, Max\u202f!',
 'Cool, Raoul\u202f!',
 'Du calme\u202f!',
 'Relaxe !',
 'Ca

In [10]:
len(target_characters)

72

In [11]:
input_characters= sorted(list(input_characters))
target_characters= sorted(list(target_characters))
num_encoder_tokens= len(input_characters)
num_decoder_tokens= len(target_characters)
max_encoder_seq_length= max([len(txt) for txt in input_characters])
max_decoder_seq_length= max([len(txt) for txt in target_characters])

In [12]:
print('No of samples: ' , len(input_texts))
print('No of unique input tokens: ', num_encoder_tokens)
print('No of unique output token: ', num_decoder_tokens)
print('Max sequence length for inputs: ', max_encoder_seq_length)
print('Max sequence length for outputs: ', max_decoder_seq_length)

No of samples:  1000
No of unique input tokens:  58
No of unique output token:  72
Max sequence length for inputs:  1
Max sequence length for outputs:  1


In [13]:
# assigning token to every characters
input_token_index= dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index= dict([(char, i) for i, char in enumerate(target_characters)])

In [14]:
input_token_index, target_token_index

({' ': 0,
  '!': 1,
  '$': 2,
  "'": 3,
  ',': 4,
  '.': 5,
  '1': 6,
  '5': 7,
  '9': 8,
  '?': 9,
  'A': 10,
  'B': 11,
  'C': 12,
  'D': 13,
  'E': 14,
  'F': 15,
  'G': 16,
  'H': 17,
  'I': 18,
  'J': 19,
  'K': 20,
  'L': 21,
  'M': 22,
  'N': 23,
  'O': 24,
  'P': 25,
  'R': 26,
  'S': 27,
  'T': 28,
  'U': 29,
  'W': 30,
  'Y': 31,
  'a': 32,
  'b': 33,
  'c': 34,
  'd': 35,
  'e': 36,
  'f': 37,
  'g': 38,
  'h': 39,
  'i': 40,
  'j': 41,
  'k': 42,
  'l': 43,
  'm': 44,
  'n': 45,
  'o': 46,
  'p': 47,
  'q': 48,
  'r': 49,
  's': 50,
  't': 51,
  'u': 52,
  'v': 53,
  'w': 54,
  'x': 55,
  'y': 56,
  'z': 57},
 {' ': 0,
  '!': 1,
  "'": 2,
  ',': 3,
  '-': 4,
  '.': 5,
  '1': 6,
  '9': 7,
  '?': 8,
  'A': 9,
  'B': 10,
  'C': 11,
  'D': 12,
  'E': 13,
  'F': 14,
  'G': 15,
  'H': 16,
  'I': 17,
  'J': 18,
  'L': 19,
  'M': 20,
  'N': 21,
  'O': 22,
  'P': 23,
  'Q': 24,
  'R': 25,
  'S': 26,
  'T': 27,
  'U': 28,
  'V': 29,
  'a': 30,
  'b': 31,
  'c': 32,
  'd': 33,
  'e': 

In [15]:
# one hot encoding using numpy
# syntax(num of pair, max eng sentence len, no of eng char)
encoder_input_data= np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data= np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data= np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [23]:
# onehot representation
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1
    
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by 1 timestep
        decoder_input_data[i, t, target_token_data[char]]= 1
        
        if t>0:
            # decoder_target_data will be ahead of 1 timestep and not include start charracter
            decoder_target_data[i, t-1, target_token_index[char]] =1
            
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1
    decoder_input_data[i, t:, target_token_index[' ']] = 1

KeyError: 'V'

In [21]:
encoder_input_data[0].shape

(1, 58)

In [27]:
# define input sequence and process it
encoder_inputs= Input(shape=(None, num_encoder_tokens))
encoder= LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c= encoder(encoder_inputs)
# discard encoder_outputs and keep the states
encoder_states= [state_h, state_c]

In [28]:
# set up decoder, using encoder_states as initial states
decoder_inputs= Input(shape=(None, num_decoder_tokens))
decoder_lstm= LSTM(latent_dim, return_sequences= True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state= encoder_states)
decoder_dense= Dense(num_decoder_tokens, activation='softmax')
decoder_outputs= decoder_dense(decoder_outputs)

In [29]:
# defining model which turn encoder_input_data & decoder_input_data into decoder_outouts
model= Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [30]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/100
13/13 [==============================] - 13s 204ms/step - loss: 0.0000e+00 - accuracy: 0.0800 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/100
13/13 [==============================] - 0s 34ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/100
13/13 [==============================] - 0s 36ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/100
13/13 [==============================] - 0s 34ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/100
13/13 [==============================] - 0s 33ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/100
13/13 [==============================] - 0s 34ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/100
13/13 [==============================] - 0

13/13 [==============================] - 0s 33ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 53/100
13/13 [==============================] - 0s 32ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 54/100
13/13 [==============================] - 0s 33ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 55/100
13/13 [==============================] - 0s 32ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 56/100
13/13 [==============================] - 0s 32ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 57/100
13/13 [==============================] - 0s 37ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 58/100
13/13 [==============================] - 0s 35

In [36]:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state 
# 3) repeat with current target taken and current states

# define sample model
encoder_model= Model(encoder_inputs, encoder_states)

decoder_state_input_h= Input(shape=(latent_dim,))
decoder_state_inputs_c= Input(shape=(latent_dim,))
decoder_state_inputs= [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c= decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
decoder_states= [state_h, state_c]
decoder_outputs= decoder_dense(decoder_outputs)

decoder_model= Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

# reverse lookup token index to decode sequence back to make it readable
reverse_input_char_index= dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index= dict((i, char) for char, i in target_token_index.items())

In [48]:
def decode_sequence(input_seq):
    # encode input as state vectors
    state_value= encoder_model.predict(input_seq)
    
    # generate empty target sequence of length 1
    target_seq= np.zeros((1,1, num_decoder_tokens))
    
    # populate the first characters of target sequence
    target_seq[0,0, target_token_index['\t']] = 1
    
    # sampling loop for a batch of sequence
    stop_condition= False
    decode_sentence= ' '
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # sample a token
        sampled_token_index= np.argmax(output_tokens[0, -1, :])
        sampled_char= reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        # exit condition: either hit max length or fins=d stop char
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition= True
        
        # update target sequence of len 1
        target_seq= np.zeros((1,1, num_decoder_tokens))
        target_seq[0,0, sampled_token_index] =1
        
        # update status
        states_value= [h,c]
        
    return decoded_sentence

In [49]:
for seq_index in range(100):
    input_seq= encoder_input_data[seq_index: seq_index + 1]
    decoded_Sentence= decode_sequence(input_seq)
    print('-')
    print('Input sentence: ', input_texts[seq_index])
    print('Decoded sentence: ', decoded_sentence)

KeyError: '\t'